<a href="https://colab.research.google.com/github/e19166/e19-4yp-Dynamic-Multi-Dimensional-Resource-Orchestration-in-Kubernetes/blob/main/Models/SDG/Resource_SDG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas scikit-learn optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 13.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
import optuna

In [8]:
# Load and preprocess the dataset
df = pd.read_csv("/content/service-1-deployment_dataset.csv")

In [9]:
# --- Feature Engineering ---

# Convert Timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed')

# Sort by time
df = df.sort_values('Timestamp')

# Time features
df['hour'] = df['Timestamp'].dt.hour
df['dayofweek'] = df['Timestamp'].dt.dayofweek

# Usage percentages
df['cpu_usage_pct'] = df['CPU Usage'] / df['CPU Limit']
df['memory_usage_pct'] = df['Memory Usage'] / df['Memory Limit']
df['request_rate_rps'] = df['Request Rate']

# Rolling trends (example over 5 past rows, you can adjust window)
df['cpu_usage_mean_5'] = df['cpu_usage_pct'].rolling(window=5).mean().fillna(method='bfill')
df['memory_usage_mean_5'] = df['memory_usage_pct'].rolling(window=5).mean().fillna(method='bfill')
df['cpu_usage_std_5'] = df['cpu_usage_pct'].rolling(window=5).std().fillna(method='bfill')
df['memory_usage_std_5'] = df['memory_usage_pct'].rolling(window=5).std().fillna(method='bfill')

# Targets: CPU Needed and Memory Needed = CPU Usage & Memory Usage of next timestamp
df['cpu_needed_t+1'] = df['CPU Usage'].shift(-1)
df['memory_needed_t+1'] = df['Memory Usage'].shift(-1)

# Drop NaN due to rolling/shift
df = df.dropna()

/tmp/ipython-input-9-4281671913.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['cpu_usage_mean_5'] = df['cpu_usage_pct'].rolling(window=5).mean().fillna(method='bfill')
/tmp/ipython-input-9-4281671913.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['memory_usage_mean_5'] = df['memory_usage_pct'].rolling(window=5).mean().fillna(method='bfill')
/tmp/ipython-input-9-4281671913.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['cpu_usage_std_5'] = df['cpu_usage_pct'].rolling(window=5).std().fillna(method='bfill')
/tmp/ipython-input-9-4281671913.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['memory_usage_std_